# FRQ 2
> Relationship of FRQ 2 to Project Based learning.
- title: FRQ2 - Classes
-- toc: true
- badges: false
- image: /images/frqs.png
- categories: [1.B]
- tags: [api]
- type: ap
- week: 13

## Classes Background
> All Classes FRQs relate well to POJOs in PBL.  The 2019 FRQ #2 describes a problem of a Step Tracker.  The task is to write a StepTracker class.   Included is the constructor and any required instance variables and
methods.
- Assumption would be a integer or long steps instance variable, also it implies data to track days.
- Constructor: StepTracker tr = new StepTracker(10000);
- Get Methods with algorithms: tr.activeDays(); tr.averageSteps(); 
- Set Methods: tr.addDailySteps(9000);

## StepTracker to PBL 
> The StepTracker would be a pretty conventional POJO in PBL. As a PBL concept, StepTracking is something that is done by an individual in relation to their personal fitness. What object or objects would you need for StepTracking...
- Person Object, in thinking about this object it seems contact information and age would be important to this application.
- Define a steps attribute that contains steps 

In [ ]:
/*
Person is a POJO, Plain Old Java Object.
 */
@Data
@NoArgsConstructor
@AllArgsConstructor
@Entity
public class Person {
    // automatic unique identifier for Person record
    @Id
    @GeneratedValue(strategy = GenerationType.AUTO)
    private Long id;

    // email, password, roles are key to login and authentication
    @NotEmpty
    @Size(min=5)
    @Column(unique=true)
    @Email
    private String email;

    @NotEmpty
    private String password;

    // @NonNull: Places this in @RequiredArgsConstructor
    @NonNull
    @Size(min = 2, max = 30, message = "Name (2 to 30 chars)")
    private String name;

    @DateTimeFormat(pattern = "yyyy-MM-dd")
    private Date dob;

    // Steps attribute
    // How is this like FRQ3?
    private ArrayList<Integer> steps = new ArrayList<>();
    
    // Initializer used when setting database from an API
    public Person(String email, String password, String name, Date dob) {
        this.email = email;
        this.password = password;
        this.name = name;
        this.dob = dob;
    }

    // A custom getter to return age from dob calculation
    // How is this like FRQ1?
    public int getAge() {
        if (this.dob != null) {
            LocalDate birthDay = this.dob.toInstant().atZone(ZoneId.systemDefault()).toLocalDate();
            return Period.between(birthDay, LocalDate.now()).getYears(); }
        return -1;
    }

    public void addSteps(int steps) {
        this.steps.add(steps);
    }

}

## Limitations
> Flaws in definition. Quickly in my analysis of this POJO definition, I started to determine there would be some flaws as the definition seems to simple to track days.  For instance...
- How could application determine relationship of Day of Week, Month, Year that steps were taken
- How could application update steps after initial add them initially.

### Collection, List, ArrayList
> The usage of Collection, List, or ArrayList enables a POJO to reference other POJOs.  Often in Java you will see these different names, here is brief definition...
- A Collection is an interface that defines the highest-level of shared collection behavior, and extends Iterable which allows usage of ForEach loop.
- A List is an interface that defines the highest-level of shared List behavior. The type List has in its contract that all elements have an order, that the order will not change except through intended manipulation.  Key interfaces: A List allows get() the n'th element;  A List defines add() to put new elements at the end. 
- ArrayList is an implementation of List.  ArrayList should not be used in an API. Its usage should only be used in implementation detail.

## Hacks
> Build your own POJOs.   In the POJOs you will have a relationship.  For instance, if you redefine the Person POJO to be multiple POJOs, Person and Day.   The POJO Person and the  POJO Day contain relationships.  List is the interface and ArrayList is the implementation.  The Person data with Day would be able to track and change the List of Day(s).  
- [Read this article for Background](https://vladmihalcea.com/the-best-way-to-map-a-onetomany-association-with-jpa-and-hibernate/).

In [ ]:
/*
"1 to Many" and "Many to 1"
- 1 Person to Many Day(s) Relationship (Forward)
- Many Days to 1 Person Relationship (Reverse)
- symbol "-|" means the Day to Person must contain a result
- symbol "O<-" means there are zero to many Day(s) were data is collected
-----------              -----------
| Person  | -|-----O<- | Day     |  
-----------              -----------
*/

// Relationship shown, data is incomplete
@Entity
public class Person {
    // Person data

    // relationship
    @OneToMany(
        cascade = CascadeType.ALL,
        orphanRemoval = true
    )
    private List<Day> days = new ArrayList<>();
}

// Relationship shown, data is incomplete
@Entity
public class Day {
    // attributes, how would you track days.  perhaps capture date and have many method to calculate this data
    private int year;
    private int day;
    private int steps;
 
    // relationship
    @ManyToOne(fetch = FetchType.LAZY)
    private Person person;
}